# Motif specification
Each motif string contains a pdb id, specification of motif amino acid chain and residue indexes, and specification of which of these residues may be redesigned.

# Example motifs
* "7F7P,B32-46;A7-21,B32-46;A7-21" --> two segments of motif. Residues 32 through 46 on chain B and residues 7 through 21 on chain A.  All positions may be redesigned.

* "4xoj,A55;A99;A190-192,A191" --> three segments of motif.  All on chain A.  Residues 55, 99, 190, 191, and 192.  Only residue 191 may be redesigned.

# Outputs
The output of the processing is a pdb file with the coordinates of motif residues.
* Each segment of the motif is labeled as its own chain, in order A, B, C, etc.
* For that may be redesigned, all atoms other than N, CA, C and O are removed and the residue type is set to UNK.
* The header of the motif includes a contig specifying how the motif is placed in the native scaffold.
  * Example:  consider "4xoj,A55;A99;A190-192,A191" the header contig should be "38;A;43;B;90;C;46"
  * 4xoj has 223 resolved residues (indexed as 16 through 238), the 38 corresponds to the 38 residues 16-54 before residue 55 (segment A), the 43 corresponds to the residues between residue 55 and 99 and so on. The final 46 indicates that the native structure terminates with 46 additional residues that are not part of the motif.

In [1]:
import os
import urllib.request
import Bio
import numpy as np
from Bio.PDB import PDBParser, Select
motif_chain_id_order = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [2]:
def parse_contig_string(contig_string):
    contig_segments = []
    for motif_segment in contig_string.split(";"):
        segment_dict ={"chain":motif_segment[0]}
        if "-" in motif_segment:
            segment_dict["start"], segment_dict["end"] = motif_segment[1:].split("-")

        else:
            segment_dict["start"] = segment_dict["end"] = motif_segment[1:]
        contig_segments.append(segment_dict)
    return contig_segments

def check_if_in_segment(redesign_segments, chain, residue):
    for segment in redesign_segments:
        if segment["chain"] == chain:
            if int(segment["start"]) <= residue <= int(segment["end"]):
                return True
    return False

class AltLocSelect(Select):
    def accept_atom(self, atom):
        # Accept atoms with altloc 'A' or no altloc
        return atom.altloc in ('A', ' ')

def remove_alt_conformations(structure):
    # Remove atoms with alternate location identifiers other than 'A'
    for model in structure:
        for chain in model:
            for residue in chain:
                # Create a list of atoms to remove
                atoms_to_remove = []
                for atom in residue:
                    #if atom.altloc not in ('A', ' '):
                    if atom.get_full_id()[4][1] not in ('A', ' '):
                        atoms_to_remove.append(atom)
                    elif atom.altloc == 'A':
                        atom.set_altloc(' ')
                if atoms_to_remove:
                    for atom in atoms_to_remove:
                        residue.detach_child(atom.id)
    return structure

                



In [3]:
# Fetch the PDB file from RCSB and parse it with BioPython
def load_pdb(pdb_id, native_pdb_dir="./native_pdbs/"):
    pdb_fn = f"{native_pdb_dir}{pdb_id}.pdb" # File name to save the PDB file to
    urllib.request.urlretrieve(
        f'http://files.rcsb.org/download/{pdb_id}.pdb',
        f"{native_pdb_dir}{pdb_id}.pdb")

    # Parse the PDB file and extract the motif segments.
    # Then save a new pdb file with each segment concatenated, with each segment in a separate chain.
    parser = PDBParser()
    structure = parser.get_structure(pdb_id, f"{native_pdb_dir}{pdb_id}.pdb")
    return structure


def parse_motif_into_new_structure(structure, motif_segments, redesign_segments):
    # Create a new structure object to hold the motif segments
    motif_structure = Bio.PDB.Structure.Structure("motif")
    motif_structure.add(Bio.PDB.Model.Model(0))

    # Iterate through segments creating a new chain for each segment 
    for i, segment in enumerate(motif_segments):
        motif_structure[0].add(Bio.PDB.Chain.Chain(motif_chain_id_order[i]))

        # Iterate through residues in the segment and add them to the new chain.
        for residue in structure[0][segment["chain"]]:
            if int(segment["start"]) <= residue.id[1] <= int(segment["end"]):

                # Remove all hydrogen atoms.
                for atom in list(residue):
                    if atom.element == "H":
                        residue.detach_child(atom.id)

                # Check if the residue is in a redesign segment.
                # If so, remove atoms except the backbone and set type to UNK.
                if check_if_in_segment(redesign_segments, segment["chain"], residue.id[1]):
                    for atom in list(residue):
                        if atom.id not in ["N", "CA", "C", "O"]:
                            residue.detach_child(atom.id)
                    residue.resname = "UNK"
                
                # Reset residue index to start from 1.
                residue = residue.copy()
                residue.id = (" ", residue.id[1]-int(segment["start"])+1, " ")

                # Add the residue to the new chain.
                motif_structure[0][motif_chain_id_order[i]].add(residue)

    return motif_structure


# Center the motif structure on the origin.
# Because of some BioPython weirdness, we have to do this manually,
# by reading the saved pdb file and calculating the center of mass,
# then translating the structure to the origin, and saving it again.
def center_pdb(motif_fn):
    # Load the motif structure from the saved pdb file.
    parser = PDBParser()
    motif_structure = parser.get_structure("motif", motif_fn)


    ### Translate the motif structure to the origin.
    # Get the center of mass of the motif structure.
    com = np.array([0., 0., 0.])
    count = 0
    for atom in motif_structure.get_atoms():
        atom_vec = atom.get_vector().get_array()
        com += atom_vec
        count += 1
    com = com/count

    # Translate the motif structure to the origin.
    for atom in motif_structure.get_atoms():
        atom.set_coord(atom.get_coord() - com)

    # Save the centered motif structure, overwriting the original file.
    io = Bio.PDB.PDBIO()
    io.set_structure(motif_structure)
    io.save(motif_fn)

In [11]:
def build_contig_string(structure, motif_segments):
    # Write a contig string describing the placement of the motif in the original structure, restricting to resolved residues.

    # First find the first resolved residue in the structure.
    first_residue = None
    motif_chain = motif_segments[0]["chain"]
    for residue in structure[0][motif_chain]:
        if residue.id[1] != " ":
            first_residue = residue.id[1]
            break

    # Iterate through the motif segments and compute the number of residues between them.
    contig_string = ""
    for i, segment in enumerate(motif_segments):
        chain = motif_chain_id_order[i]
        if i == 0:
            contig_string += f"{int(segment['start']) - first_residue};{chain};"
        else:
            contig_string += f"{int(segment['start']) - int(motif_segments[i-1]['end']) - 1};{chain};"
        if i == len(motif_segments) - 1:
            break

    # get index of last residue in the structure, excluding heteroatoms
    chain = segment["chain"]
    last_AA_idx = 0
    for residue in structure[0][chain]:
        if residue.id[0] == " ": # check that it is not a heteroatom
            last_AA_idx = residue.id[1]

    contig_string += f"{last_AA_idx - int(segment['end'])}"
    return contig_string


def save_motif_pdb(motif_string, motif_pdb_dir="./motif_pdbs/", native_pdb_dir="./native_pdbs/"):
    pdb_id, motif_residues, redesign_residues = motif_string.split(",")

    # Load structure and parse motif into new structure
    structure = load_pdb(pdb_id, native_pdb_dir=native_pdb_dir)

    # Some pdbs have alternate conformations, which we remove.
    structure = remove_alt_conformations(structure)

    motif_segments = parse_contig_string(motif_residues)
    redesign_segments = parse_contig_string(redesign_residues) if redesign_residues else []
    motif_structure = parse_motif_into_new_structure(structure, motif_segments, redesign_segments)

    # Save the new structure to a PDB file with a header that includes the place of the motif in the original structure.
    contig_string = build_contig_string(structure, motif_segments)
    header_string = f"REMARK 1 Original PDB ID: {pdb_id}\n"
    header_string += f"REMARK 2 Contig: {contig_string}\n"

    motif_fn = f"{motif_pdb_dir}{pdb_id}_motif.pdb"
    io = Bio.PDB.PDBIO()
    io.set_structure(motif_structure)
    io.save(motif_fn , AltLocSelect(), write_end=True)

    # Center the motif structure on the origin.
    center_pdb(motif_fn)

    # Prepend header_string to the new PDB file by copying the file and writing the header first.
    with open(f"{motif_pdb_dir}{pdb_id}_motif.pdb", "r") as f:
        file_string = f.read()
    with open(f"{motif_pdb_dir}{pdb_id}_motif.pdb", "w") as f:
        f.write(header_string + file_string)


In [12]:
motif_specs_paths = [
    "../test_cases/rfdiffusion_benchmark/motif_specs.csv",
    "../test_cases/orphans/motif_specs.csv",
    "../test_cases/other_enzymes/motif_specs.csv"
]

# Make sure the motif_pdb_dir and native_pdb_dir directories exist.
motif_pdb_dir = "../motif_pdbs/"
if not os.path.exists(motif_pdb_dir): os.makedirs(motif_pdb_dir)

native_pdb_dir = "../native_pdbs/"
if not os.path.exists(native_pdb_dir): os.makedirs(native_pdb_dir)

# Parse the motif specs files and save the motif PDBs.
for motif_specs_path in motif_specs_paths:
    with open(motif_specs_path, "r") as f:
        for line in f:
            print(line.strip())
            save_motif_pdb(line.strip(), motif_pdb_dir=motif_pdb_dir, native_pdb_dir=native_pdb_dir)

1BCF,A18-25;A47-54;A92-99;A123-130,A19-25;A47-50;A52-53;A92-93;A95-99;A123-126;A128-129


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 17010.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 17055.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 17057.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 17102.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 17104.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

5TPN,A163-181,A163-168;A170-171;A179
5IUS,A119-140;A63-82,A63;A65;A67;A69;A71-72;A76;A79-80;A82;A119-123;A125;A127;A129-130;A133;A135;A137-138;A140


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 11147.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 11154.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 11160.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 11173.
  warnings.warn(


3IXT,P254-277,P255;P258-259;P262-263;P268;P271-272;P275-276


...

/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2410.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2506.
  warnings.warn(


7MQQ,A115-129,
7MQQ,A80-94;A115-129,
7S5L,A27-41;A77-91,
7WRK,A80-94,
7WRK,A99-113;A132-146,
6cpa,A69-72;A127;A196;A248;A270,A70-71
1mpy,A153;A199;A214;A246;A255;A265,


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10615.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10620.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 10625.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 10630.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10635.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder

1b73,A7-8;A70;A178-180,A179
2rkx,A9-11;A48-50;A101;A128;A169;A176;A201;A222-224,A10;A49;A223
3b5v,A51-53;A81;A110;A131;A159;A180-184;A210-211;A231-233,A52;A181;A183;A232
4xoj,A55;A99;A190-192,A191


/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4413.
  warnings.warn(
/Users/briantrippe/opt/anaconda3/lib/python3.9/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5285.
  warnings.warn(
